# Neuer Abschnitt

In [ ]:
# Die Pandas-Bibliothek wird hier als "pd" importiert.
# Pandas ist eine leistungsstarke Bibliothek zur Datenverarbeitung und -analyse. Sie erleichtert
# das Laden, Bearbeiten und Speichern von tabellenartigen Daten, z. B. aus CSV-Dateien.
import pandas as pd

# Die Transformers-Bibliothek bietet eine Sammlung vortrainierter Modelle für verschiedene NLP-Aufgaben,
# wie Textklassifikation, Textgenerierung und Übersetzung. Hier wird speziell die "pipeline"-Funktion importiert,
# die eine vereinfachte Möglichkeit bietet, vortrainierte Modelle zu verwenden.
from transformers import pipeline

# NLTK (Natural Language Toolkit) ist eine Bibliothek zur Sprachverarbeitung, die eine Vielzahl an
# Tools und Datensätzen für natürliche Sprachverarbeitung (Natural Language Processing, NLP) enthält,
# wie z. B. Tokenizer, Wortlisten und Korpora.
import nltk

# Die Module `DataLoader` und `Dataset` aus `torch.utils.data` dienen zur Handhabung von Datensätzen.
# "Dataset" bietet eine Standardmethode zur Definition eines Datensatzes, während "DataLoader" Funktionen
# zum effizienten Laden, Shuffeln und Batchen der Daten bereitstellt. Diese Module sind zentral für das Training von Modellen.
from torch.utils.data import DataLoader, Dataset

# Die PyTorch-Bibliothek wird hier importiert, die eine der populärsten Bibliotheken für Deep Learning ist.
# Sie bietet grundlegende Werkzeuge für die Arbeit mit neuronalen Netzen, Tensoren und GPU-unterstütztem Rechnen.
import torch

# Hier werden Module für neuronale Netzwerke und die Optimierung in PyTorch importiert.
# `torch.nn` enthält Bausteine für den Aufbau neuronaler Netze,
# während `torch.optim` Werkzeuge für die Optimierung von Gewichten im Trainingsprozess bereitstellt.
import torch.nn as nn
import torch.optim as optim

# `LabelEncoder` wird aus sklearn.preprocessing importiert.
# Dieser Encoder wird verwendet, um kategorische Labels (z.B. "positive", "negative") in numerische Werte umzuwandeln,
# da maschinelle Lernmodelle nur numerische Daten verarbeiten können.
from sklearn.preprocessing import LabelEncoder

# `train_test_split` aus `sklearn.model_selection` trennt die Daten in Trainings- und Testdatensätze.
# Dies ist eine Standardtechnik, um die Modellleistung zu testen und Überanpassung (Overfitting) zu vermeiden.
from sklearn.model_selection import train_test_split

# Die Funktion `pad` aus `torch.nn.functional` wird importiert. Sie dient dazu, Eingaben mit unterschiedlicher Länge
# auf eine einheitliche Länge zu bringen, indem sie am Ende der Sequenzen Padding-Zeichen einfügt.
# Dies ist besonders nützlich für Textdaten, da diese in der Regel variierende Längen aufweisen.
from torch.nn.functional import pad

# `BertForSequenceClassification` ist ein vortrainiertes BERT-Modell aus der Transformers-Bibliothek,
# das speziell für Klassifikationsaufgaben angepasst ist. Es kann verwendet werden, um Textsequenzen
# in Kategorien einzuordnen, nachdem es auf spezifische Daten trainiert wurde.
from transformers import BertForSequenceClassification

# Der `BertTokenizer` wird verwendet, um Eingabetexte in Token umzuwandeln, die das Modell verstehen kann.
# Tokenizer teilen Text in einzelne Bestandteile (Tokens) auf und konvertieren sie in IDs,
# die als Eingabe für das BERT-Modell dienen.
from transformers import BertTokenizer

# Import des BERT-Modells und des Tokenizers aus der Transformers-Bibliothek
# `BertModel` ist das BERT-Sprachmodell, das kontextuelle Darstellungen von Text erstellt.
# `BertTokenizer` konvertiert Texte in Token und Token-IDs, die BERT als Eingabe verarbeitet.
from transformers import BertModel, BertTokenizer

# Installation der benötigten Bibliotheken
!pip install transformers

# Download von NLTK-Ressourcen
nltk.download('wordnet')
nltk.download('omw-1.4')

# Laden der notwendigen Python Bibliotheken, um mit Datensatz aus Github zu arbeiten
import os
import requests
import pandas as pd

# Festlegen des aktuellen Verzeichnisses
current_directory = os.getcwd()

# URL der manuell annotierten FAQs-Liste auf GitHub
# file_url = "https://raw.githubusercontent.com/username/repo/main/FAQs-Liste.csv"
# Hinweis: Ersetzen Sie die URL mit der tatsächlichen URL Ihrer FAQs-Liste auf GitHub

file_name = "FAQs - Liste.xlsx"

# Die FAQs-Liste von GitHub herunterladen und lokal speichern
# response = requests.get(file_url)
# with open(file_name, "wb") as file:
# file.write(response.content)

# Datei in ein Pandas DataFrame laden
# Anpassung der Einleseparameter basierend auf dem Format Ihrer CSV-Datei


# Anzeigen der geladenen Daten

# Prüfen, ob die benötigten Spalten vorhanden sind
required_columns = ['Frage', 'Antwort', 'Thema']

df = pd.read_excel(file_name, engine='openpyxl', usecols=required_columns)
df = df.dropna(subset=required_columns)
print(df.head())

def split_by_topic(df, test_samples_per_topic=2, random_state=42):
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    for topic in df['Thema'].unique():
        topic_df = df[df['Thema'] == topic]
        topic_test_df = topic_df.sample(n=test_samples_per_topic, random_state=random_state)
        topic_train_df = topic_df.drop(topic_test_df.index)
        train_df = pd.concat([train_df, topic_train_df])
        test_df = pd.concat([test_df, topic_test_df])
    return train_df, test_df

# Aufteilen der Daten in Trainings- und Testdaten
train_df, test_df = split_by_topic(df, test_samples_per_topic=2, random_state=42)

# Hier eine Funktion schreiben die beispielsweise Testdaten trennt - das heißt von jeder Frage nehme ich 2 für Test und 9 Fragen für Training

# Frage: Validation ist hoch, und Loss war niedrig - ist kein gutes Zeichn wenn Validation hoch ist aber Loss niedrig das zeigt eigentlich alles ist gut

# Festlegen der GPU oder CPU für das Training
device = torch.device('cuda')
# Definition der MultiTaskBERTClassifier-Klasse
class MultiTaskBERTClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size=768, num_answer_labels=10, num_category_labels=5):
        super(MultiTaskBERTClassifier, self).__init__()

        # Übernimmt das übergebene BERT-Modell als Basismodell für die Textrepräsentation
        self.bert = bert_model

        # Answer prediction head
        # Ein Klassifikator für die Vorhersage von Antwortkategorien
        self.answer_classifier = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, num_answer_labels)
        )

        # Category prediction head
        # Ein Klassifikator für die Vorhersage der Themenkategorie
        self.category_classifier = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, num_category_labels)
        )

    def forward(self, input_ids, attention_mask):
        # Extrahieren der CLS-Token-Darstellung
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # Repräsentation des CLS-Tokens

        # Weitergabe durch Klassifikator-Köpfe
        answer_logits = self.answer_classifier(cls_token)
        category_logits = self.category_classifier(cls_token)

        return answer_logits, category_logits



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


             Thema                                              Frage  \
1  Einkommensteuer        Muss ich den Original-Bescheid aufbewahren?   
2  Einkommensteuer            Muss ich den Originalbescheid behalten?   
3  Einkommensteuer  Ist es erforderlich, den Originalbescheid aufz...   
4  Einkommensteuer   Soll ich den Originalbescheid lange aufbewahren?   
5  Einkommensteuer  Muss ich den Originalbescheid für immer aufbew...   

    Antwort  
1  §257 HGB  
2  §257 HGB  
3  §257 HGB  
4  §257 HGB  
5  §257 HGB  


In [ ]:
# Initialisierung des BERT-Modells und des Tokenizers
model_name = 'bert-base-german-cased'
bert_model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Bestimmen der Anzahl von Antwort- und Kategorielabels
num_answer_labels = len(train_df['Antwort'].unique())
num_category_labels = len(train_df['Thema'].unique())

# Initialisierung des Multi-Task-Klassifikators
model = MultiTaskBERTClassifier(
    bert_model,
    hidden_size=768,
    num_answer_labels=num_answer_labels,
    num_category_labels=num_category_labels
).to(device)

# Definition der QADataset-Klasse
class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_len=200):
        self.tokenizer = tokenizer
        self.max_len = max_len

        # Extrahieren der Fragen, Antworten und Kategorien aus dem DataFrame
        self.questions = data['Frage'].tolist()
        self.responses = data['Antwort'].tolist()
        self.categories = data['Thema'].tolist()

        # Initialisierung der LabelEncoder für Antworten und Kategorien
        self.response_encoder = LabelEncoder()
        self.category_encoder = LabelEncoder()

        # Umwandeln der Antworten und Kategorien in numerische Labels
        self.response_labels = self.response_encoder.fit_transform(self.responses)
        self.category_labels = self.category_encoder.fit_transform(self.categories)

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        response_label = self.response_labels[idx]
        category_label = self.category_labels[idx]

        # Tokenisierung der Frage mit dem BERT-Tokenizer
        encoding = self.tokenizer.encode_plus(
            question,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        # Rückgabe der tokenisierten Frage und der Labels als Tensoren
        return input_ids, attention_mask, torch.tensor(response_label, dtype=torch.long), torch.tensor(category_label, dtype=torch.long)

# Erstellen der Datasets und DataLoader für Training und Test
train_dataset = QADataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

test_dataset = QADataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Definition der Verlustfunktionen
criterion_answer = nn.CrossEntropyLoss()
criterion_category = nn.CrossEntropyLoss()

# Initialisierung des Optimierers
optimizer = optim.Adam(model.parameters(), lr=1e-5) # vorher 2le-5 Hyperparameter Anpassung

# Trainingsfunktion
def train_multi_task_model(model, dataloader, criterion_answer, criterion_category, optimizer, num_epochs=3):
    model.train()  # Setzt das Modell in den Trainingsmodus

    for epoch in range(num_epochs):
        epoch_loss = 0
        correct_answers = 0
        correct_categories = 0
        total = 0

        for batch in dataloader:
            input_ids, attention_mask, answer_labels, category_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            answer_labels = answer_labels.to(device)
            category_labels = category_labels.to(device)

            optimizer.zero_grad()

            # Vorwärtsdurchlauf
            answer_outputs, category_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Verlustberechnung
            loss_answer = criterion_answer(answer_outputs, answer_labels)
            loss_category = criterion_category(category_outputs, category_labels)
            loss = loss_answer + loss_category

            # Backward-Pass
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # Berechnen der Genauigkeit
            _, predicted_answers = torch.max(answer_outputs, 1)
            _, predicted_categories = torch.max(category_outputs, 1)

            total += answer_labels.size(0)
            correct_answers += (predicted_answers == answer_labels).sum().item()
            correct_categories += (predicted_categories == category_labels).sum().item()

        # Genauigkeit für die Epoche berechnen
        epoch_answer_accuracy = correct_answers / total
        epoch_category_accuracy = correct_categories / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}, Answer Accuracy: {epoch_answer_accuracy:.4f}, Category Accuracy: {epoch_category_accuracy:.4f}")

    return model


# Training des Modells durchführen
print("Starte Training...")
model = train_multi_task_model(model, train_loader, criterion_answer, criterion_category, optimizer, num_epochs=10) # Hier vorher 20

# Evaluationsfunktion
def evaluate_multi_task_model(model, dataloader, criterion_answer, criterion_category):
    model.eval()  # Setzt das Modell in den Evaluationsmodus
    eval_loss = 0
    correct_answers = 0
    correct_categories = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, answer_labels, category_labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            answer_labels = answer_labels.to(device)
            category_labels = category_labels.to(device)

            # Vorwärtsdurchlauf
            answer_outputs, category_outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            # Verlustberechnung
            loss_answer = criterion_answer(answer_outputs, answer_labels)
            loss_category = criterion_category(category_outputs, category_labels)
            loss = loss_answer + loss_category
            eval_loss += loss.item()

            # Berechnen der Genauigkeit
            _, predicted_answers = torch.max(answer_outputs, 1)
            _, predicted_categories = torch.max(category_outputs, 1)

            total += answer_labels.size(0)
            correct_answers += (predicted_answers == answer_labels).sum().item()
            correct_categories += (predicted_categories == category_labels).sum().item()

    # Ausgabe der Evaluationsergebnisse
    avg_loss = eval_loss / len(dataloader)
    answer_accuracy = correct_answers / total
    category_accuracy = correct_categories / total
    print(f"Validation Loss: {avg_loss:.4f}, Answer Accuracy: {answer_accuracy:.4f}, Category Accuracy: {category_accuracy:.4f}")

    return answer_accuracy, category_accuracy

# Evaluation des trainierten Modells
print("Evaluiere Modell...")
answer_accuracy, category_accuracy = evaluate_multi_task_model(model, test_loader, criterion_answer, criterion_category)

# Speichern der LabelEncoder für spätere Vorhersagen
# Erstelle neue LabelEncoder-Instanzen und trainiere sie mit allen verfügbaren Daten
label_encoder_answer = LabelEncoder()
label_encoder_answer.fit(df['Antwort'].unique())

label_encoder_category = LabelEncoder()
label_encoder_category.fit(df['Thema'].unique())

# Mapping der Themenkategorien zu Mitarbeitern
mitarbeiter_map = {
    'Lohn': 'Alexandra Himmelreich',
    'EDV': 'Luke Horchler',
    'FiBu': 'Sven Althaus',
    'Unternehmensberatung': 'Stephan Sonneborn'
}

# Funktion zur Vorhersage mit Unsicherheitsbehandlung
def predict_and_handle_uncertainty(model, question, true_answer="", true_category="", threshold=0.4):
    model.eval()
    encoding = tokenizer.encode_plus(
        question,
        truncation=True,
        max_length=200,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        # Vorhersage durchführen
        answer_logits, category_logits = model(input_ids=input_ids, attention_mask=attention_mask)

        # Softmax-Wahrscheinlichkeiten berechnen
        answer_probs = torch.softmax(answer_logits, dim=1)
        category_probs = torch.softmax(category_logits, dim=1)

        # Maximale Wahrscheinlichkeit und zugehörigen Index bestimmen
        answer_confidence, answer_pred = torch.max(answer_probs, dim=1)
        category_confidence, category_pred = torch.max(category_probs, dim=1)

    # Vorhersagen in lesbare Ausgaben umwandeln
    if answer_confidence.item() < threshold:
        answer_pred_label = "Unable to provide a confident answer:", label_encoder_answer.inverse_transform(answer_pred.cpu().numpy())[0]
    else:
        answer_pred_label = label_encoder_answer.inverse_transform(answer_pred.cpu().numpy())[0]

    if category_confidence.item() < threshold:
        category_pred_label = "Unable to determine category"
    else:
        category_pred_label = label_encoder_category.inverse_transform(category_pred.cpu().numpy())[0]

    # Ausgabe der Vorhersageergebnisse
    print(f"Predicted Answer: {answer_pred_label}")
    print(f"Predicted Category: {category_pred_label}")

    if true_answer and true_category:
        print(f"Correct Answer: {true_answer}")
        print(f"Correct Category: {true_category}")

    print(f"Answer Prediction Confidence: {answer_confidence.item()}")
    print(f"Category Prediction Confidence: {category_confidence.item()}")

    # Entscheidung über die Ausgabe basierend auf Konfidenzwerten
    message = ""
    if answer_confidence.item() < threshold and category_confidence.item() < threshold:
        message = "Leider kann ich das nicht bestimmen. Bitte wenden Sie sich an unser Support-Team für Unterstützung."
    elif answer_confidence.item() < threshold and category_confidence.item() >= threshold:
        if category_pred_label in mitarbeiter_map:
            message = f"Bitte wenden Sie sich an {mitarbeiter_map[category_pred_label]}!"
        else:
            message = "Leider kann ich das nicht bestimmen. Bitte wenden Sie sich an unser Support-Team für Unterstützung."
    else:
        message = answer_pred_label

    print(message + "\n")
    return message

# Beispielhafte Vorhersagen
print("\nBeispielhafte Vorhersagen:")

# Speichern des trainierten Modells
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'num_answer_labels': num_answer_labels,
    'num_category_labels': num_category_labels
}, 'faq_model.pth')

print("Modell gespeichert.")

# Speichern der LabelEncoder
import pickle
with open('label_encoder_answer.pkl', 'wb') as f:
    pickle.dump(label_encoder_answer, f)
with open('label_encoder_category.pkl', 'wb') as f:
    pickle.dump(label_encoder_category, f)

print("LabelEncoder gespeichert.")

# Beispielhafte Vorhersagen
question = "Was ist bei Einstellung eines Werksstudenten zu beachten?"
predict_and_handle_uncertainty(model, question,"Werkstudenten, die neben ihrem Studium arbeiten, sind in der Regel versicherungsfrei in der Kranken-, Pflege- und Arbeitslosenversicherung, wenn sie in der vorlesungsfreien Zeit nicht mehr als 20 Stunden pro Woche arbeiten. Sie sind jedoch in der Rentenversicherung pflichtversichert. Lohnsteuerrechtlich sollte die Besteuerung nach der Lohnsteuerkarte des Studenten erfolgen, und der Student muss sich selbst um eine studentische Krankenversicherung kümmern.", "Lohn")

question = "Was ist ein Unternehmens-Check?"
predict_and_handle_uncertainty(model, question, "Der L&P Unternehmens-Check ist der Einstieg in ein präzises Analyse- und Steuerungssystem für deinen Betrieb. Verbinde eine Standortbestimmung durch die Spezialisten von Lückel & Partner mit der Einführung digitaler Prozesse in deinem Unternehmen. Spezialisten liefern verständliche und aussagekräftige Zahlen und begleiten dich für einen nachhaltigen Erfolg in deinem Unternehmen. https://www.lup-beratung.de/unternehmens-check-holzbau", "Unternehmensberatung")

question = "Darf der Urlaub im Bauhauptgewerbe bei Austritt an den Arbeitnehmer ausgezahlt werden?"
predict_and_handle_uncertainty(model, question, "Nein, der Urlaub wird mitgenommen zum neuen Arbeitgeber. Sollte der Mitarbeiter nicht mehr im Baugewerbe tätig werden, kann er nach 3 Monaten die Urlaubsabgeltung bei der Soka-Bau beantragen.", "Lohn")

print("Fertig.")

Starte Training...
Epoch [1/10], Loss: 5.7184, Answer Accuracy: 0.0288, Category Accuracy: 0.6224
Epoch [2/10], Loss: 4.6916, Answer Accuracy: 0.1416, Category Accuracy: 0.9327
Epoch [3/10], Loss: 4.0511, Answer Accuracy: 0.3129, Category Accuracy: 0.9843
Epoch [4/10], Loss: 3.4114, Answer Accuracy: 0.5830, Category Accuracy: 0.9939
Epoch [5/10], Loss: 2.7877, Answer Accuracy: 0.7622, Category Accuracy: 0.9956
Epoch [6/10], Loss: 2.2374, Answer Accuracy: 0.8907, Category Accuracy: 0.9974
Epoch [7/10], Loss: 1.7588, Answer Accuracy: 0.9510, Category Accuracy: 0.9991
Epoch [8/10], Loss: 1.3855, Answer Accuracy: 0.9694, Category Accuracy: 0.9991
Epoch [9/10], Loss: 1.0957, Answer Accuracy: 0.9755, Category Accuracy: 0.9991
Epoch [10/10], Loss: 0.8741, Answer Accuracy: 0.9851, Category Accuracy: 0.9991
Evaluiere Modell...
Validation Loss: 6.3234, Answer Accuracy: 0.0000, Category Accuracy: 1.0000

Beispielhafte Vorhersagen:
Modell gespeichert.
LabelEncoder gespeichert.
Predicted Answer: We